In [1]:
import os
import json
from tqdm import tqdm

from process import compress_video

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense

### 비디오 데이터 전처리 및 npy로 저장

In [19]:
import cv2
import numpy as np

def preprocess_video_every_3_seconds(video_path:str, frame_size:tuple, block_nums:int, frame_rate=3):
    """
    Extracts frames every 3 seconds from a video file, resizing them to frame_size and converting to grayscale.
    
    Args:
    video_path (str): Path to the video file.
    frame_size (tuple): Size (height, width) to resize frames.
    block_nums (int) : Total count for three-seconds-blocks
    frame_rate (int): Number of frames to extract per second within the 3-second window.

    Returns:
    List[numpy.ndarray]: List of sequences, where each sequence is a numpy array of shape (num_frames, height, width, 1).
    """

    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    interval = int(fps * 3)

    sequences = []
    while True:
        frames = []
        for _ in range(interval):
            success, frame = vidcap.read()
            if not success:
                break
            frame = cv2.resize(frame, frame_size, interpolation=cv2.INTER_AREA)
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = np.expand_dims(gray_frame, axis=-1)  # 채널 수 늘려줌
            gray_frame = gray_frame.astype(np.float32) / 255.0 
            frames.append(gray_frame)

        if len(frames) == 0:
            break
        ##### 지금 초당 9프레임으로 잡혀있음 (frame_rate*3을 frame_rate으로 바꿈)
        if len(frames) >= frame_rate : 
            sequences.append(np.array(frames[:frame_rate * 3]))  # 모든 frame이 3초단위로 들어갈 수 있도록 제어
        
        if len(sequences) > block_nums:
            break

    vidcap.release()
    return np.array(sequences[:-1])


In [ ]:
def parse_annotations(annotations:list):
    """
    Extracts Every Annotation from json label file
    
    Args:
    annotations(List): List of Dictionary for annotations label with highlight and represent

    Returns:
    Dict: Whether each block is Highlight or not
    """
    highlight_map = {}
    
    for annot in annotations:
        block_num = annot['highlight']
        for num in block_num:
            highlight_map[num] = 1
            
    ret = [0] * len(highlight_map)
    for i, item in enumerate(highlight_map.items()):
        ret[item] = 1
                
    return highlight_map

In [6]:
# video_length = ['2~5분', '5~20분']
video_length = ['2~5분']
output_video_dir = 'processed/video/'

In [9]:
for i, leng in enumerate(video_length):

    output_json_path = f'processed/label/processed_video_data_{i}.json'
    json_path = f'data/라벨링데이터/video_summary_validation_data({leng}).json'
    video_path = f'data/원천데이터/{leng}/'

    with open(json_path, 'r', encoding='utf-8') as f:
        label_data = json.load(f)

    new_video_data = []
    video_idx = 1

    for item in tqdm(label_data):
        input_video_name = item['filename'] + '.npy'
        input_video_path = os.path.join(video_path, input_video_name)
        output_video_name = f"processed_video_{video_idx}.npy"
        output_video_path = os.path.join(output_video_dir, output_video_name)

        if not os.path.exists(input_video_path):
            print(f"Not Found : {input_video_path}")
            continue

        ########
        # 영상 전처리 진행 및 저장
        # 처리할 것 : three_secs json 에서 불러와서 마지막 숫자 번호 보고 그 길이로 축소시키기
        blocks_num = item["three_secs"][-1]
        annotations = item['anoots']

        output = preprocess_video_every_3_seconds(input_video_path, (256, 256), blocks_num)
        np.save(output_video_path, output)

        category = item["category"]

        item['filename'] = output_video_name
        item['category'] = category.encode('utf-8').decode()
        item['path'] = output_video_path
        item['quality'] = '256 256' # 추 후에 데이터 사용할 때, split으로 사용할 수 있게 띄워쓰기로 구분

        video_idx += 1
        new_video_data.append(item)

    # 전처리된 데이터에 대해 라벨을 새로 저장해줌
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(new_video_data, f, ensure_ascii=False, indent=2)

    print(f"Process Finish :: {leng}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/idaeho/Documents/GitHub/project_shorts/data/라벨링데이터/2~5분/SUMVID_SHORT_TRAIN/video_summary_training_data(2~5분).json'